In [1]:
#!/gpfs01/bartels/user/vplikat/anaconda3/bin/python

##########
# HEADER #
##########
# The data for this script is derived from an fMRI experiment in which
# subjects viewed different videos. Either magic, control or surprise videos. 
# The experiment was devided into 3 blocks. Each block consisted of 4 
# experimental runs. In each run subjects viewed 24 videos (each video is 
# considered a trial).
# The videos in each block were associated with one object (Balls, Cards and 
# Sticks) and there were 3 magic effects (Appear, Change and Vanish). For each
# magic effect and object there are two trick versions (i.e. Appear1, Appear2,
# Change1,...). This resulted in 6 magic videos per object = 18 magic videos 
# and for every magic video there was a corresponding control video showing
# the same movements without the magical effect. Additionally per object there
# were 3 surprise videos showing unusual surprising actions performed with the
# objects (e.g. eating a playing card).
# After the second run in each block the underlying method behind each magic 
# trick was presented.

#                       TIME
#   ---------------------------------->
#   OBJECT1 R1  R2  Revelation  R3  R4  |
#   OBJECT2 R1  R2  Revelation  R3  R4  |   TIME
#   OBJECT3 R1  R2  Revelation  R3  R4  v

# RUNS: 2*Appear1 Magic 2*Appear2 Magic 2*Appear1 Control 2*Appear2 Control
#       2*Vanish1 Magic 2*Vanish2 Magic 2*Vanish1 Control 2*Vanish2 Control
#       2*Change1 Magic 2*Change2 Magic 2*Change1 Control 2*Change2 Control
#       2*Surprise1     2*Surprise2     2*Surprise13
#       = 24 Videos

# The aim of the experiment was to find neural correlates of surprise and in 
# particular of surprising events we consider "impossible". 
# The data used are beta estimate NIfTI images derived from a GLM using SPM12 
# in MATLAB. 

##########################
# PURPOSE OF THIS SCRIPT #
##########################
# The purpose of this script is to look into the eyetracking data and check 
# if subjects fixated at the same position during the special moment pre vs post
# revelation

################################
# FUNCTIONALITY OF THIS SCRIPT #
################################
# Additionally move the edf files to sourcedata
# FIRST STEP 
# Import all libraries needed and get all important path information
# SECOND STEP 
# Eyetracking data preprocessing:
# a) remove data before and after blinks
# b) interpolate missing data (blinks and other)
# c) low pass and high pass filter of diameter
# d) demean pupil diameter per condition
# e) divide by standard deviation per session (run in this case)
# f) resample - all data 
# THIRD STEP
# Save preprocessed data in 'derivatives' folder

In [2]:
# FIRST STEP
# interact with the operating system 
import os
from pathlib import Path
import git
import glob
import csv
# data structuration and calculations
import pandas as pd  # to create data frames
import numpy as np   # most important numerical calculations
from scipy import signal
from scipy.spatial import distance
from scipy import stats
from lmfit import minimize, Parameters, Parameter, report_fit
from itertools import combinations
# importing torch to test stuff written by Peter
import torch
# read in mat files
import readmat
# needed to extract the run number out of the parentesis of the string in the SPM.mat file
import re
# optimize time performance
import time
# plotting
import matplotlib.pyplot as plt
# video processing
import cv2
from my_ET_functions import eyedata2pandasframe

In [8]:
def pixel_per_degree (pixWidth,dist2Monitor):
    deg = math.degrees((.5*pixWidth)/dist2Monitor)
    deg_per_pixel = 2*np.arctan (deg)
    pixel_per_deg = 1/deg_per_pixel
    return pixel_per_deg
interval = 6            # in the paper they use 6... 
x = np.linspace(0, interval, interval*int(fps))

def single_pupil_IRF(params, x):
    s1 = params['s1']
    n1 = params['n1']
    tmax1 = params['tmax1']
    return s1 * ((x**n1) * (np.e**((-n1*x)/tmax1)))

params = Parameters()
params.add('s1', value=-8.969)
params.add('n1', value=3.737)
params.add('tmax1', value=0.917)

def double_pupil_IRF(params, x):
    s1 = params['s1']
    s2 = params['s2']
    n1 = params['n1']
    n2 = params['n2']
    tmax1 = params['tmax1']
    tmax2 = params['tmax2']
    return s1 * ((x**n1) * (np.e**((-n1*x)/tmax1))) + s2 * ((x**n2) * (np.e**((-n2*x)/tmax2)))

params2 = Parameters()
params2.add('s1', value=-6661)
params2.add('s2', value=0.823)
params2.add('n1', value=7.751)
params2.add('n2', value=17.061)
params2.add('tmax1', value=0.844)
params2.add('tmax2', value=2.540)


blink_kernel = double_pupil_IRF(params2, x)
sac_kernel = single_pupil_IRF(params, x)

In [25]:
################################################
# VARIABLES FOR PATH SELECTION AND DATA ACCESS #
################################################
HOME            = str(Path.home())
# DATA
PROJ_DIR        = os.path.join(HOME, 'Documents', 'Magic_fMRI', 'DATA', 'MRI')
STIM_DIR = os.path.join(PROJ_DIR,'stimuli')
RAW_DIR         = os.path.join(PROJ_DIR, 'rawdata')
DERIVATIVES_DIR = os.path.join(PROJ_DIR, 'derivatives')
DATA_DIR = os.path.join(DERIVATIVES_DIR, 'eyetracking')
subjects = glob.glob(os.path.join(DATA_DIR, 'sub*'))
subjects.sort()
subjects = [os.path.basename(sub) for sub in subjects]
# ANALYSIS

MAGIC_MOMENT_MAT = os.path.join(PROJ_DIR,'code','analysis', 'glm', 'info_MagicMoment_merged.mat')

In [33]:
info_magic_moment = readmat.load(MAGIC_MOMENT_MAT,isStruct=True)['do']
videos = info_magic_moment['ListOfVideos']
special_frame = info_magic_moment['all_frames_of_effect']

fps = 25
spf = 1/fps
pres_frame_width = 1600
dist_to_monitor = 1050 #mm

all_runs = np.linspace(1,12,12,dtype=int)
pre_post_runs = (all_runs-1)//2
pre_runs = all_runs[pre_post_runs%2==0]
post_runs = all_runs[pre_post_runs%2==1]

flipping_runs = [1,4,5,8,9,12]
#flipping_runs = [1,3,5,7,9,11]

slack_time = 0.25

In [24]:
vid = 'Stick_Vanish2_Magic'
special_frame[videos.index(vid)]
sub = subjects[12]

event_dfs = glob.glob(os.path.join(RAW_DIR,sub,'func','*_events.tsv'))
event_dfs.sort()
et_datafiles = glob.glob(os.path.join(RAW_DIR,sub,'func','*recording-eyetracking_physio.asc'))
et_datafiles.sort()

if len(event_dfs)!=len(et_datafiles):
    print('Sub not used')
    
for idx,(event,et) in enumerate(zip(event_dfs,et_datafiles)):
    curr_run = pd.read_csv(event, sep='\t')
    if not any(curr_run.trial_type.str.contains(vid)):
        continue
    

using run 5
using run 6
using run 7
using run 8
using run 9
using run 10
using run 11
using run 12


In [4]:
# loop over videos
vid = 'Stick_Vanish2_Magic'

matrices = []
fisher_matrices = []

# loop over subjects
for s, sub in enumerate(SUBJECS):
    event_files = glob.glob(os.path.join(RAW_DIR,os.path.basename(sub),'func','*_events.tsv'))
    ET_files = glob.glob(os.path.join(DATA_DIR,os.path.basename(sub),'*_recording-eyetracking_physio_preprocessed.tsv'))

    event_files.sort()
    ET_files.sort()

    if len(event_files) != len(ET_files):
        print (sub +' is missing some files')
        continue

    positions_x = []
    positions_y = []
    for event, ET in zip(event_files, ET_files):
        run = list(map(int, re.findall(r'\d+', os.path.basename(event))))[1]

        event_df = pd.read_csv(event,sep='\t')
        ET_data = pd.read_csv(ET, sep='\t')

        indices_of_interest = [i for i,trial in enumerate(event_df.trial_type) if vid in trial]
        for index in indices_of_interest:
            pos_x = ET_data.X_Coord[(ET_data.TimeStamp>=event_df.onset[index]) &
                                   (ET_data.TimeStamp<=event_df.rating_onset[index])].values
            pos_y = ET_data.Y_Coord[(ET_data.TimeStamp>=event_df.onset[index]) &
                                   (ET_data.TimeStamp<=event_df.rating_onset[index])].values
            pos = ET_data[['X_Coord','Y_Coord']][(ET_data.TimeStamp>=event_df.onset[index]) &
                                   (ET_data.TimeStamp<=event_df.rating_onset[index])].values

            flip = ('_F' in event_df.trial_type[index] and run in flipping_runs) or (
                '_F' not in event_df.trial_type[index] and run not in flipping_runs)
            if flip:
                pos_x = abs(pos_x - pres_frame_width)
            positions_x.append(pos_x)
            positions_y.append(pos_y)

    df_x=pd.DataFrame(list(map(np.ravel, positions_x))).T
    df_y=pd.DataFrame(list(map(np.ravel, positions_y))).T

    corr_matrix = (df_x.corr()+df_y.corr())/2
    # do fisher transformation and append the correlation matrix to the array
    matrices.append(corr_matrix)
    fisher_matrices.append(np.arctanh(corr_matrix))

/gpfs01/bartels/user/vplikat/Documents/Master_Thesis/DATA/MRI/derivatives/eyetracking/sub-04 is missing some files
/gpfs01/bartels/user/vplikat/Documents/Master_Thesis/DATA/MRI/derivatives/eyetracking/sub-05 is missing some files


In [7]:
sub_dict = {
    'vids': [],
    'pre_revelation': [],
    'runs': [],
    'type': [],
    'x_positions': [],
    'y_positions': []
}

for event, ET in zip(event_files, ET_files):
    run = list(map(int, re.findall(r'\d+', os.path.basename(event))))[1]
    
    event_df = pd.read_csv(event,sep='\t')
    ET_data = pd.read_csv(ET, sep='\t')
    
    # iterate over videos
    for index,row in event_df.iterrows():
        vid = row.trial_type
        flip = ('_F' in vid and run in flipping_runs) or ('_F' not in vid and run not in flipping_runs)
        
        if '_F' in vid:
            vid = vid[:-2]
            
        special_moment = special_frame[videos==vid][0]*spf + row.onset
        
        x_pos = ET_data.X_Coord[(ET_data.TimeStamp>=special_moment-slack_time) & 
                                (ET_data.TimeStamp<=special_moment+slack_time)].values
        y_pos = ET_data.Y_Coord[(ET_data.TimeStamp>=special_moment-slack_time) & 
                                (ET_data.TimeStamp<=special_moment+slack_time)].values
        
        x = np.nanmean(x_pos)
        if flip:
            x = abs(x-frame_width)

        y = np.nanmean(y_pos)
        
        sub_dict['x_positions'].append(x)
        sub_dict['y_positions'].append(y)
        sub_dict['vids'].append(vid)
        sub_dict['pre_revelation'].append(run in pre_runs)
        sub_dict['runs'].append(run)
        if 'Magic' in vid:
            sub_dict['type'].append('Magic')
        elif 'Control' in vid:
            sub_dict['type'].append('Control')
        elif 'Surprise' in vid:
            sub_dict['type'].append('Surprise')
        else:
            raise
        
sub_df = pd.DataFrame(sub_dict,columns=sub_dict.keys())

In [26]:
event = event_files[0]
ET = ET_files[0]

run = list(map(int, re.findall(r'\d+', os.path.basename(event))))[1]
event_df = pd.read_csv(event,sep='\t')
ET_data = pd.read_csv(ET, sep='\t')

vid = event_df.trial_type[0]
vid_start = event_df.onset[0]

if '_F' in vid:
    flip = True
    vid = vid[:-2]
else:
    flip = False
    
special_moment = special_frame[videos==vid][0]*spf + vid_start

In [112]:
# loop over videos
vid = 'Stick_Appear2_Magic'

# loop over subjects
s = 0
sub = os.path.basename(SUBJECS[s])

# loop over run files
event_files = glob.glob(os.path.join(RAW_DIR,sub,'func','*_events.tsv'))
ET_files = glob.glob(os.path.join(DATA_DIR,sub,'*_recording-eyetracking_physio_preprocessed.tsv'))

event_files.sort()
ET_files.sort()

if len(event_files) != len(ET_files):
    print (sub +' is missing some files')
    raise
    
positions_x = []
positions_y = []
positions = []
for event, ET in zip(event_files, ET_files):
    run = list(map(int, re.findall(r'\d+', os.path.basename(event))))[1]
    
    event_df = pd.read_csv(event,sep='\t')
    ET_data = pd.read_csv(ET, sep='\t')
    
    indices_of_interest = [i for i,trial in enumerate(event_df.trial_type) if vid in trial]
    for index in indices_of_interest:
        pos_x = ET_data.X_Coord[(ET_data.TimeStamp>=event_df.onset[index]) &
                               (ET_data.TimeStamp<=event_df.rating_onset[index])].values
        pos_y = ET_data.Y_Coord[(ET_data.TimeStamp>=event_df.onset[index]) &
                               (ET_data.TimeStamp<=event_df.rating_onset[index])].values
        pos = ET_data[['X_Coord','Y_Coord']][(ET_data.TimeStamp>=event_df.onset[index]) &
                                             (ET_data.TimeStamp<=event_df.rating_onset[index])].values
        
        flip = ('_F' in event_df.trial_type[index] and run in flipping_runs) or (
            '_F' not in event_df.trial_type[index] and run not in flipping_runs)
        if flip:
            pos_x = abs(pos_x - pres_frame_width)
        positions_x.append(pos_x)
        positions_y.append(pos_y)
        positions.append(pos)
        
df_x=pd.DataFrame(list(map(np.ravel, positions_x))).T
df_y=pd.DataFrame(list(map(np.ravel, positions_y))).T

corr_matrix = (df_x.corr()+df_y.corr())/2